In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pickle # To save model

In [40]:
# Random seed for consistency
np.random.seed(42)

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [3]:
# Label Encoder use to Encode target labels with value between 0 and n_classes-1
Encoder = LabelEncoder()

In [4]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [5]:
# TfidfVectorizer Convert a collection of raw documents to a matrix of TF-IDF features.
Tfidf_vect = TfidfVectorizer()

In [6]:
data = pd.read_csv("dataset/Data_latih.csv")
data.head(5)
data['label'].value_counts()

label
1    3465
0     766
Name: count, dtype: int64

In [7]:
data2 = pd.read_csv("dataset/politik_merge.csv")
data2.head(5)

,Judul,Waktu,Link,Content,tag1,tag2,tag3,tag4,tag5,source
0,Jokowi Kenakan Pakaian Adat Betawi di Sidang T...,16/08/2024,https://nasional.kompas.com/read/2024/08/16/11...,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me...",Presiden Jokowi,Jokowi,sidang tahunan MPR RI 2024,Jokowi adat Betawi sidang mpr 2024,Megawati tak hadiri sidang tahunan MPR 2024,kompas
1,Amnesty International Beberkan 6 Indikator Kri...,2024-07-18,https://nasional.tempo.co/read/1893144/amnesty...,"TEMPO.CO, Jakarta - Amnesty International Indo...",Amnesty International,Amnesty International Indonesia,Kebebasan Berpendapat,Indeks Demokrasi,Revisi UU TNI,tempo
2,"Jelang Long Weekend, Stasiun Kereta Cepat Hali...","Rabu, 08 Mei 2024 19:18 WIB",https://news.detik.com/berita/d-7331666/jelang...,"Stasiun kereta cepat Whoosh di Halim, Jakarta ...",kereta cepat whoosh,stasiun halim,long weekend,NaN,NaN,detik
3,KPU Tegaskan Pemilih Tak Terdaftar di DPT Bisa...,13/02/2024,https://nasional.kompas.com/read/2024/02/13/21...,"JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (...",KPU,pemilu 2024,Hasyim Asy'ari,NaN,NaN,kompas
4,Kemenag Luncurkan Gerakan Senam Haji Jaga Keta...,2024-04-29,https://nasional.tempo.co/read/1861810/kemenag...,"TEMPO.CO, Jakarta - Kementerian Agama atau Kem...",Senam Haji,Kemenag,Jemaah Haji,Ibadah Haji,Asrama Haji,tempo


In [8]:
df1 = pd.DataFrame(data)
df2 = pd.DataFrame(data2)

In [9]:
df1.head(5)

,ID,label,tanggal,judul,narasi,nama file gambar
0,71,1,17-Aug-20,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,71.jpg
1,461,1,17-Jul-20,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png
2,495,1,13-Jul-20,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png
3,550,1,8-Jul-20,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png
4,681,1,24-Jun-20,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg


In [10]:
df1 = df1.drop(["ID",'tanggal','nama file gambar'], axis='columns')

In [11]:
df1.head(5)

,label,judul,narasi
0,1,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...
1,1,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...
2,1,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...
3,1,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k..."
4,1,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .


In [12]:
#merging
df1['berita'] = df1['judul']+df1['narasi']


In [13]:
df1.head(5)

,label,judul,narasi,berita
0,1,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,Pemakaian Masker Menyebabkan Penyakit Legionna...
1,1,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,Instruksi Gubernur Jateng tentang penilangan ...
2,1,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,Foto Jim Rohn: Jokowi adalah presiden terbaik ...
3,1,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...","ini bukan politik, tapi kenyataan Pak Jokowi b..."
4,1,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,Foto Kadrun kalo lihat foto ini panas dinginKa...


In [14]:
df1.drop(['judul','narasi'], axis='columns')

,label,berita
0,1,Pemakaian Masker Menyebabkan Penyakit Legionna...
1,1,Instruksi Gubernur Jateng tentang penilangan ...
2,1,Foto Jim Rohn: Jokowi adalah presiden terbaik ...
3,1,"ini bukan politik, tapi kenyataan Pak Jokowi b..."
4,1,Foto Kadrun kalo lihat foto ini panas dinginKa...
...,...,...
4226,1,: KPK Dilarang Membawa Brimob Bersenjata Masuk...
4227,1,Foto pejabat keuangan dibawah palu aritJangan ...
4228,1,Gambar Denny Siregar Musuh Warga Tasikmalaya d...
4229,1,Kaesang: Bapak Saya dengan Kesederhaan Bisa Ni...


In [15]:
df1['label'].value_counts()

label
1    3465
0     766
Name: count, dtype: int64

In [16]:
df2 = df2.drop(['Waktu','Link','tag1','tag2','tag3','tag4','tag5','source'],axis='columns')

In [17]:
df2.head(5)

,Judul,Content
0,Jokowi Kenakan Pakaian Adat Betawi di Sidang T...,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me..."
1,Amnesty International Beberkan 6 Indikator Kri...,"TEMPO.CO, Jakarta - Amnesty International Indo..."
2,"Jelang Long Weekend, Stasiun Kereta Cepat Hali...","Stasiun kereta cepat Whoosh di Halim, Jakarta ..."
3,KPU Tegaskan Pemilih Tak Terdaftar di DPT Bisa...,"JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (..."
4,Kemenag Luncurkan Gerakan Senam Haji Jaga Keta...,"TEMPO.CO, Jakarta - Kementerian Agama atau Kem..."


In [18]:
df2['berita'] = df2['Judul']+df2['Content']
df2.head(5)

,Judul,Content,berita
0,Jokowi Kenakan Pakaian Adat Betawi di Sidang T...,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me...",Jokowi Kenakan Pakaian Adat Betawi di Sidang T...
1,Amnesty International Beberkan 6 Indikator Kri...,"TEMPO.CO, Jakarta - Amnesty International Indo...",Amnesty International Beberkan 6 Indikator Kri...
2,"Jelang Long Weekend, Stasiun Kereta Cepat Hali...","Stasiun kereta cepat Whoosh di Halim, Jakarta ...","Jelang Long Weekend, Stasiun Kereta Cepat Hali..."
3,KPU Tegaskan Pemilih Tak Terdaftar di DPT Bisa...,"JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (...",KPU Tegaskan Pemilih Tak Terdaftar di DPT Bisa...
4,Kemenag Luncurkan Gerakan Senam Haji Jaga Keta...,"TEMPO.CO, Jakarta - Kementerian Agama atau Kem...",Kemenag Luncurkan Gerakan Senam Haji Jaga Keta...


In [19]:
df2 = df2[:4000]

In [20]:
df2.tail(5)

,Judul,Content,berita
3995,Khofifah Usul Evakuasi Anak Palestina ke Jatim...,Menteri Pertahanan Prabowo Subianto menceritak...,Khofifah Usul Evakuasi Anak Palestina ke Jatim...
3996,Ketum PBNU Singgung Dirinya dan Gus Dur juga P...,"TEMPO.CO, Jakarta - Ketua Umum Pengurus Besar ...",Ketum PBNU Singgung Dirinya dan Gus Dur juga P...
3997,Ketika Nama Kaesang Disebut-sebut sebagai Calo...,"TEMPO.CO, Jakarta - Nama Ketua Umum Partai Sol...",Ketika Nama Kaesang Disebut-sebut sebagai Calo...
3998,"Suami Kimberly Absen Pemeriksaan Polisi, Minta...",Polisi mengagendakan pemeriksaan terhadap Edwa...,"Suami Kimberly Absen Pemeriksaan Polisi, Minta..."
3999,"Tidak Ada Kompromi, Orangtua Korban Enggan Dam...","JAKARTA, KOMPAS.com - Irfan Maulana, kuasa hu...","Tidak Ada Kompromi, Orangtua Korban Enggan Dam..."


In [21]:
df2['label'] = 0

In [22]:
df2.drop(['Judul','Content'], axis='columns')

,berita,label
0,Jokowi Kenakan Pakaian Adat Betawi di Sidang T...,0
1,Amnesty International Beberkan 6 Indikator Kri...,0
2,"Jelang Long Weekend, Stasiun Kereta Cepat Hali...",0
3,KPU Tegaskan Pemilih Tak Terdaftar di DPT Bisa...,0
4,Kemenag Luncurkan Gerakan Senam Haji Jaga Keta...,0
...,...,...
3995,Khofifah Usul Evakuasi Anak Palestina ke Jatim...,0
3996,Ketum PBNU Singgung Dirinya dan Gus Dur juga P...,0
3997,Ketika Nama Kaesang Disebut-sebut sebagai Calo...,0
3998,"Suami Kimberly Absen Pemeriksaan Polisi, Minta...",0


In [23]:
frames = [df1,df2]

In [24]:
df = pd.concat(frames)
df['label'].value_counts()

label
0    4766
1    3465
Name: count, dtype: int64

In [25]:
feature = df['berita'].astype(str)
label = df['label'].astype(int)

In [26]:
feature

0       Pemakaian Masker Menyebabkan Penyakit Legionna...
1       Instruksi Gubernur Jateng tentang penilangan  ...
2       Foto Jim Rohn: Jokowi adalah presiden terbaik ...
3       ini bukan politik, tapi kenyataan Pak Jokowi b...
4       Foto Kadrun kalo lihat foto ini panas dinginKa...
                              ...                        
3995    Khofifah Usul Evakuasi Anak Palestina ke Jatim...
3996    Ketum PBNU Singgung Dirinya dan Gus Dur juga P...
3997    Ketika Nama Kaesang Disebut-sebut sebagai Calo...
3998    Suami Kimberly Absen Pemeriksaan Polisi, Minta...
3999    Tidak Ada Kompromi, Orangtua Korban Enggan Dam...
Name: berita, Length: 8231, dtype: object

In [27]:
label

0       1
1       1
2       1
3       1
4       1
       ..
3995    0
3996    0
3997    0
3998    0
3999    0
Name: label, Length: 8231, dtype: int64

In [29]:
# melakukan stemming pada setiap baris
lower = [stemmer.stem(row.lower()) for row in feature]

# Hasil stem dan lower
lower[:5]

['pakai masker sebab sakit legionnairesa caller to a radio talk show recently shared that his wife was hospitalized n told she had covid n only a couple of days left to live a doctor friend suggested she be tested for legionnaires disease because she wore the same mask every day all day long turns out it was legionnaires disease from the moisture n bacteria in her mask she was given antibiotics n within two days was better what if these spikes in covid are really something else due to mask induced infections',
 'instruksi gubernur jateng tentang tilang bagi yg tidak masker di muka umum rp 150 000 guna e-tilang via apps pikobaryth seluruh anggota grup sesuai instruksi gubernur jawa tengah hasil rapat tim gugus tugas covid 19 jateng sbb',
 'foto jim rohn jokowi adalah presiden baik dlm sejarah bangsa indonesiajokowi adalah presiden baik dlm sejarah bangsa indonesia jim rohn motivator baik dunia',
 'ini bukan politik tapi nyata pak jokowi hasil pulang 11 000 triliun uang negara dari swiss

In [41]:
# Melakukan tokenisasi untuk setiap baris dataset
tokens = [word_tokenize(element) for element in lower]

# Hasil tokenisasi
tokens[:5]

[['pakai',
  'masker',
  'sebab',
  'sakit',
  'legionnairesa',
  'caller',
  'to',
  'a',
  'radio',
  'talk',
  'show',
  'recently',
  'shared',
  'that',
  'his',
  'wife',
  'was',
  'hospitalized',
  'n',
  'told',
  'she',
  'had',
  'covid',
  'n',
  'only',
  'a',
  'couple',
  'of',
  'days',
  'left',
  'to',
  'live',
  'a',
  'doctor',
  'friend',
  'suggested',
  'she',
  'be',
  'tested',
  'for',
  'legionnaires',
  'disease',
  'because',
  'she',
  'wore',
  'the',
  'same',
  'mask',
  'every',
  'day',
  'all',
  'day',
  'long',
  'turns',
  'out',
  'it',
  'was',
  'legionnaires',
  'disease',
  'from',
  'the',
  'moisture',
  'n',
  'bacteria',
  'in',
  'her',
  'mask',
  'she',
  'was',
  'given',
  'antibiotics',
  'n',
  'within',
  'two',
  'days',
  'was',
  'better',
  'what',
  'if',
  'these',
  'spikes',
  'in',
  'covid',
  'are',
  'really',
  'something',
  'else',
  'due',
  'to',
  'mask',
  'induced',
  'infections'],
 ['instruksi',
  'gubernur'

In [42]:
print(tokens[2])

['foto', 'jim', 'rohn', 'jokowi', 'adalah', 'presiden', 'baik', 'dlm', 'sejarah', 'bangsa', 'indonesiajokowi', 'adalah', 'presiden', 'baik', 'dlm', 'sejarah', 'bangsa', 'indonesia', 'jim', 'rohn', 'motivator', 'baik', 'dunia']


In [43]:
X_train, X_test, y_train, y_test = train_test_split(tokens, label, test_size=0.2, stratify=label)

In [44]:
len(X_train)

6584

In [45]:
len(X_test)

1647

In [46]:
# Encoder for Data Label
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

y_train

array([0, 0, 0, ..., 0, 0, 1], shape=(6584,))

In [47]:
# Fitting dataset terhadap tf-idf
Tfidf_vect.fit(["".join(row) for row in X_train])

TfidfVectorizer()

In [48]:
# Save the vectorizer using pickle
with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(Tfidf_vect, file)

In [49]:
# Mentransformasikan hasil fitting terhadap data X_train dan X_test
X_train_Tfidf = Tfidf_vect.transform([" ".join(row) for row in X_train])
X_test_Tfidf = Tfidf_vect.transform([" ".join(row) for row in X_test])

In [50]:
model = MultinomialNB()
model.fit(X_train_Tfidf, y_train)

akurasi = model.score(X_test_Tfidf, y_test)
print("Akurasi dari model :"+str((akurasi)*100))

Akurasi dari model :64.35944140862173


In [51]:
y_pred = model.predict(X_test_Tfidf)
report = classification_report(y_test,y_pred)

In [52]:
print(y_pred)

[0 1 0 ... 0 0 0]


In [53]:
print(report)

              precision    recall  f1-score   support

           0       0.62      0.99      0.76       954
           1       0.92      0.17      0.28       693

    accuracy                           0.64      1647
   macro avg       0.77      0.58      0.52      1647
weighted avg       0.75      0.64      0.56      1647



In [54]:
len(y_test)*0.94554

1557.30438

In [55]:
with open("mytext.txt","w", encoding="utf-8") as f:
  f.write(feature.iloc[500])

In [56]:
with open("mytext.txt","r",encoding="utf-8") as f:
  text=f.read()

In [57]:
text

'fadlizon mendaulat Anies jadi Duta GermoTUKANG BO’ONG JADI DUTA GERMO? Demi mewujudkan janji saat kampanye “Jakarta Bermaksiat”, @aniesbaswedan segera menyiapkan ‘New Normal’ untuk Diskotek dan Panti Pijat. Atas prestasi spektakuler membuka Diskotek dan Panti Pijat yang dipuja kaum kadrun kadal gurun ini, @fadlizon mendaulat Anies jadi Duta Germo.'

In [58]:
vectorized_ip = Tfidf_vect.transform([text])

In [59]:
output_array = model.predict(vectorized_ip)
result = int(output_array[0])
if(result==1):
  print("Teks tersebut merupakan berita hoax!")
else:
  print("Teks tersebut merupakan berita tidak hoax!")

Teks tersebut merupakan berita tidak hoax!


In [60]:
# Save the model to a file
with open('model_berita.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)